<a href="https://colab.research.google.com/github/Phaneendraaa/RAG_YT/blob/main/yt_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install openai-whisper yt-dlp
import os
import whisper
import subprocess



In [18]:
def download_audio(yt_url,outputfile="audio1.mp3"):
  command = ["yt-dlp","-x","--audio-format","mp3","-o",outputfile,yt_url]
  subprocess.run(command,check=True)
  return outputfile

def transcribe_audio(audio_file):
  model = whisper.load_model("base")
  result = model.transcribe(audio_file)
  return result["text"]

In [19]:
vedio_url = "https://youtu.be/bbz2boNSeL0?si=q9rWoYBiqrPjXTeE"
audio_file = "audio1.mp3"
download_audio(vedio_url,audio_file)
transcript = transcribe_audio(audio_file)
print(transcript)
if os.path.exists(audio_file):
  os.remove(audio_file)
  print(f"Deleted: {audio_file}")
else:
  print(f"File not found: {audio_file}")
with open("transcript.txt", "w", encoding="utf-8") as f:
  f.write(transcript)
print("Transcript saved to transcript.txt")

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 I see something. I see something. Danajaya Hiliarachi. You and I are not very different from this flower, just like this flower is unique. You are unique. All of us have something special that makes us as beautiful. Do you know what makes you special? Now the answer to that can be a little difficult to find. Because sometimes life has a cruel way of picking out your petals. Breaking you into and throwing you into the trash. And when you are broken, it's very difficult to feel special. Mr. Contascha, my fellow flowers. I can remember the first time I broke. I was 17 years old. I had already flunked high school and managed to get myself arrested. Now, I was afraid of the cops, but there was one person that was very afraid of. And that was my mama. Raise your hand if you have an emotional mother. Let me see. Put them all together. You get my mama. I can hear her scream outside the police station. Even the cops were afraid. She came up to me. Health, the iron bars. Looked into my eyes. An

In [20]:
with open("transcript.txt", "r", encoding="utf-8") as f:
  current_transcript = f.read()
  print("\nTranscript Preview:\n", current_transcript, "...\n")


Transcript Preview:
  I see something. I see something. Danajaya Hiliarachi. You and I are not very different from this flower, just like this flower is unique. You are unique. All of us have something special that makes us as beautiful. Do you know what makes you special? Now the answer to that can be a little difficult to find. Because sometimes life has a cruel way of picking out your petals. Breaking you into and throwing you into the trash. And when you are broken, it's very difficult to feel special. Mr. Contascha, my fellow flowers. I can remember the first time I broke. I was 17 years old. I had already flunked high school and managed to get myself arrested. Now, I was afraid of the cops, but there was one person that was very afraid of. And that was my mama. Raise your hand if you have an emotional mother. Let me see. Put them all together. You get my mama. I can hear her scream outside the police station. Even the cops were afraid. She came up to me. Health, the iron bars. L

In [21]:
!pip install sentence-transformers faiss-cpu langchain mistralai mistral-common transformers accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.8/381.8 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 72.2 MB/s eta 0:00:00


In [22]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
def load_and_split_text(file_path):
  with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()
  splitter = RecursiveCharacterTextSplitter(
      chunk_size=500,
      chunk_overlap=50,
  )
  return splitter.split_text(text)
docs = load_and_split_text("transcript.txt")


In [23]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
embed_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embed_model.encode(docs)
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [24]:
def retrieve_relevant_docs(query, top_k=3):
    query_embedding = embed_model.encode([query])
    distances, indices = index.search(np.array(query_embedding), top_k)
    return [docs[i] for i in indices[0]]


In [26]:
from mistralai import Mistral
api_key = "z6JUEWBz0GqVQvYNycWr2qINTxbTUtvt"
client = Mistral(api_key=api_key)

def answer_query_with_rag(query):
    context = "\n\n".join(retrieve_relevant_docs(query))
    messages = [
        {"role": "system", "content": "You are an assistant that answers questions based on transcripts."},
        {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {query}"}
    ]
    response = client.chat.complete(
        model="mistral-medium",
        messages=messages
    )
    return response.choices[0].message.content


In [27]:
query = "Who is the speaker?"
print(answer_query_with_rag(query))

The speaker is not explicitly identified in the provided transcript. However, based on the context, the speaker is someone who is sharing personal experiences about their life, including interactions with their father, a man named Sam, and their journey to becoming a teacher and discovering a passion for speaking. Without additional context or information, it's not possible to definitively identify the speaker.


In [28]:
processed = set()
def train_model_vedio(yt_url,file_path):
  outputfile="audio1.mp3"
  command = ["yt-dlp","-x","--audio-format","mp3","-o",outputfile,yt_url]
  subprocess.run(command,check=True)
  model = whisper.load_model("base")
  result = model.transcribe(audio_file)
  transcript=    result["text"]
  if os.path.exists(audio_file):
    os.remove(audio_file)
    print(f"Deleted: {audio_file}")
  else:
    print(f"File not found: {audio_file}")
  with open(file_path, "w", encoding="utf-8") as f:
    f.write(transcript)
  print("Transcript saved to transcript.txt")
  processed.add(yt_url)
  return file_path

def train_faiss(file_path):
  with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()
  splitter = RecursiveCharacterTextSplitter(
      chunk_size=500,
      chunk_overlap=50,
  )
  docs = splitter.split_text(text)
  embed_model = SentenceTransformer("all-MiniLM-L6-v2")
  embeddings = embed_model.encode(docs)
  dimension = embeddings.shape[1]
  index = faiss.IndexFlatL2(dimension)
  index.add(np.array(embeddings))
  query_embedding = embed_model.encode([query])
  distances, indices = index.search(np.array(query_embedding), 3)
  return [docs[i] for i in indices[0]]
def answer_query_with_rag(query,yt_url):
    if yt_url not in processed: file_path=train_model_vedio(yt_url,"transcript.txt")
    context = "\n\n".join(train_faiss("transcript.txt"))
    messages = [
        {"role": "system", "content": "You are an assistant that answers questions based on transcripts."},
        {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {query}"}
    ]
    response = client.chat.complete(
        model="mistral-medium",
        messages=messages
    )
    return response.choices[0].message.content


In [ ]:
query = "Who is the speaker?"
yt_url = ""
print(answer_query_with_rag(query,yt_url))

In [ ]:
!pip install sentence-transformers faiss-cpu langchain mistralai mistral-common transformers accelerate openai-whisper yt-dlp

In [42]:
import os
import subprocess
import whisper
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter

def train_model(video_url, transcript_path="transcript.txt", index_path="faiss_index.index"):
    audio_file = "audio.mp3"
    command = ["yt-dlp", "-x", "--audio-format", "mp3", "-o", audio_file, video_url]
    subprocess.run(command, check=True)

    model = whisper.load_model("base")
    result = model.transcribe(audio_file)
    transcript = result["text"]

    if os.path.exists(audio_file):
        os.remove(audio_file)

    with open(transcript_path, "w", encoding="utf-8") as f:
        f.write(transcript)

    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = splitter.split_text(transcript)

    embed_model = SentenceTransformer("all-MiniLM-L6-v2")
    embeddings = embed_model.encode(docs)

    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(np.array(embeddings))

    faiss.write_index(index, index_path)

    with open("processed.txt", "w") as f:
        f.write(video_url)
    processed.add(video_url)
    print("Training complete. Transcript and FAISS index saved.")

# Example:
# train_model("https://www.youtube.com/watch?v=YOUR_VIDEO_ID")


In [47]:
import os
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from mistralai import Mistral
from langchain.text_splitter import RecursiveCharacterTextSplitter

client = Mistral(api_key="z6JUEWBz0GqVQvYNycWr2qINTxbTUtvt")  # Set via env var ideally

def answer_query(query, transcript_path="transcript.txt", index_path="faiss_index.index"):
    with open(transcript_path, "r", encoding="utf-8") as f:
        text = f.read()

    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = splitter.split_text(text)

    embed_model = SentenceTransformer("all-MiniLM-L6-v2")
    query_embedding = embed_model.encode([query])

    index = faiss.read_index(index_path)
    distances, indices = index.search(np.array(query_embedding), 3)
    relevant_chunks = [docs[i] for i in indices[0]]

    context = "\n\n".join(relevant_chunks)

    messages = [
        {"role": "system", "content": "You are an assistant that answers questions based on transcripts."},
        {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {query}"}
    ]

    response = client.chat.complete(model="mistral-medium", messages=messages)
    return response.choices[0].message.content

# Example:
# print(answer_query("What was the main idea of the video?"))


In [44]:
def is_trained(video_url):
    if vedio_url in processed:
        return True
    else:
        return False


In [46]:

video_url = "https://youtu.be/Yp3ZQD5HIig?si=-_0cuQ1-7W0jCswg"
query = "is it about motivation?"
processed = set()
if not is_trained(video_url):
    train_model(video_url)

print(answer_query(query))


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


KeyboardInterrupt: 